In [28]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
from pylab import rcParams
%pylab inline
rcParams['figure.figsize'] = 12, 6
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.grid_search import GridSearchCV

Populating the interactive namespace from numpy and matplotlib


In [29]:
d = pd.read_csv('tmp/features_02.csv')
d = d[d.TripType_0 == 0]
del d['TripType_0']

In [30]:
tts = d.TripType.unique().tolist()
orig_to_new = {}
new_to_orig = {}
for i in range(len(tts)):
    orig_to_new[tts[i]] = i
    new_to_orig[i] = tts[i]

In [31]:
del d['Weekday']
del d['Weekday_nan']
lim = int(0.7 * len(d))
dtr = d[:lim]
dho = d[lim:]
tr = DataFrame({'VisitNumber': dtr.VisitNumber})
trl = DataFrame({'VisitNumber': dtr.VisitNumber})
ho = DataFrame({'VisitNumber': dho.VisitNumber})
hol = DataFrame({'VisitNumber': dho.VisitNumber})
for c in d.columns:
    if (c.startswith('ScanCount') or
        c.startswith('DepartmentDescription') or
        c.startswith('Weekday')):
        tr[c] = dtr[c].astype(float)
        ho[c] = dho[c].astype(float)
trl['TripType'] = dtr.TripType.apply(lambda t: orig_to_new[t]).astype(float)
hol['TripType'] = dho.TripType.apply(lambda t: orig_to_new[t]).astype(float)

In [34]:
xg_train = xgb.DMatrix(tr, label=trl.TripType)
xg_test = xgb.DMatrix(ho, label=hol.TripType)

In [82]:
param = {}
param['objective'] = 'multi:softmax'
param['max_depth'] = 25
param['silent'] = 1
param['n_estimators'] = 2000
param['learning_rate'] = 0.05
param['nthread'] = 4
param['num_class'] = len(tts)

watchlist = [(xg_train,'train'), (xg_test, 'test')]
num_round = 500
bst = xgb.train(param, xg_train, num_round, watchlist)
pred = bst.predict( xg_test )
print ('predicting, classification error=%f'
       % (sum( int(pred[i]) != test_Y[i] for i in range(len(test_Y))) / float(len(test_Y)) ))

[0]	train-merror:0.374700	test-merror:0.379786
[1]	train-merror:0.348151	test-merror:0.354388
[2]	train-merror:0.334428	test-merror:0.343205
[3]	train-merror:0.327410	test-merror:0.337421
[4]	train-merror:0.325902	test-merror:0.337142
[5]	train-merror:0.323752	test-merror:0.336550
[6]	train-merror:0.321856	test-merror:0.334773
[7]	train-merror:0.320825	test-merror:0.334111
[8]	train-merror:0.318839	test-merror:0.330871
[9]	train-merror:0.317690	test-merror:0.330175
[10]	train-merror:0.316958	test-merror:0.330035
[11]	train-merror:0.316077	test-merror:0.329060
[12]	train-merror:0.315196	test-merror:0.328502
[13]	train-merror:0.314345	test-merror:0.328154
[14]	train-merror:0.314136	test-merror:0.328084
[15]	train-merror:0.313240	test-merror:0.326934
[16]	train-merror:0.314390	test-merror:0.329756
[17]	train-merror:0.311971	test-merror:0.326621
[18]	train-merror:0.311986	test-merror:0.326063
[19]	train-merror:0.310433	test-merror:0.324531
[20]	train-merror:0.308163	test-merror:0.323416
[2

predicting, classification error=0.918182


In [77]:
p = DataFrame({'TripType': pred})
p.TripType = p.TripType.apply(lambda t: new_to_orig[t])
p = pd.get_dummies(p, columns=['TripType'])
p.index = pd.Index(np.arange(len(p)))
# print len(p)
# print len(p.columns)
# print p.columns
y_true = pd.get_dummies(hol.astype(int), columns=['TripType'])
y_true.index = pd.Index(np.arange(len(y_true)))
del y_true['VisitNumber']
# print len(y_true)
# print len(y_true.columns)
# print y_true.columns
p2 = DataFrame({}, index=y_true.index)
for c in y_true.columns:
    if c in p.columns:
        p2[c] = p[c]
        #print c, 'foo'
    else:
        p2[c] = np.zeros_like(y_true[c])
        #print c, 'bar'
z = y_true * p2
print z.sum().sum()
#z = z.apply(np.log)
#print z.sum().sum()
# z = z.replace([-inf], 0)
# print -z.sum().sum()/len(y_true)

169.0
